In [1]:
from pandasql import sqldf
import pandas as pd


In [102]:
df = pd.read_csv('./Cartera29122022.csv', encoding='UTF-8' )
df.head(3)

,SUC,SOCIO,NOMBRE_DEL_SOCIO,F. NACIMIENTO,SEXO,NOMBRE_ESTADO,NOMBRE_MUNICIPIO,NOMBRE_COLONIA,NOMBRE_LOCALIDAD,POBLACION,...,C VENCIDO,SCAPITAL,D. VEN,D X VEN,INTERES,MORA,DESCRIPCION_PRODUCTO,NOMBRE PROMOTOR,CRED,DESTINO
0,1,2,ISABEL CRUZ HERNANDEZ,08/07/1958,F,MORELOS,CUERNAVACA,CUERNAVACA,CUERNAVACA,"338,650",...,0.00,"45,833.33",0,12,343.74,0.00,CREDITO COLABORADORES,TEMPORAL,312,OTROS
1,1,14,CLAUDIA GONZALEZ CARDENAS,29/04/1978,F,DISTRITO FEDERAL,BENITO JUAREZ,DEL VALLE SUR,DEL VALLE SUR,"2,622",...,0.00,"30,793.63",0,47,569.68,0.00,CREDITO COLABORADORES,TEMPORAL,302,OTROS
2,1,19,GABRIELA BAUTISTA HERNANDEZ,08/03/1986,F,DISTRITO FEDERAL,LA MAGDALENA CONTRERAS,EL ROSAL,EL ROSAL,"1,710",...,0.00,"11,235.35",0,15,5.61,0.00,CREDITO COLABORADORES,TEMPORAL,301,GASTOS MEDICOS


In [103]:
df.columns

Index(['SUC', 'SOCIO', 'NOMBRE_DEL_SOCIO', 'F. NACIMIENTO', 'SEXO',
       'NOMBRE_ESTADO', 'NOMBRE_MUNICIPIO', 'NOMBRE_COLONIA',
       'NOMBRE_LOCALIDAD', 'POBLACION', 'F. EMISION', 'MONTO', 'F. VENCE',
       'T. INT', 'N. PAG', 'F. PAG', 'C VIGENTE', 'C VENCIDO', 'SCAPITAL',
       'D. VEN', 'D X VEN', 'INTERES', 'MORA', 'DESCRIPCION_PRODUCTO',
       'NOMBRE PROMOTOR', 'CRED', 'DESTINO'],
      dtype='object')

In [49]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10111 entries, 0 to 10110
Data columns (total 27 columns):
 #   Column                Non-Null Count  Dtype 
---  ------                --------------  ----- 
 0   SUC                   10111 non-null  int64 
 1   SOCIO                 10111 non-null  object
 2   NOMBRE_DEL_SOCIO      10111 non-null  object
 3   F. NACIMIENTO         10111 non-null  object
 4   SEXO                  10111 non-null  object
 5   NOMBRE_ESTADO         10111 non-null  object
 6   NOMBRE_MUNICIPIO      10111 non-null  object
 7   NOMBRE_COLONIA        10111 non-null  object
 8   NOMBRE_LOCALIDAD      10111 non-null  object
 9   POBLACION             10071 non-null  object
 10  F. EMISION            10111 non-null  object
 11  MONTO                 10111 non-null  object
 12  F. VENCE              10111 non-null  object
 13  T. INT                10111 non-null  int64 
 14  N. PAG                10111 non-null  int64 
 15  F. PAG                10111 non-null

In [119]:
df['MONTO'] = df['MONTO'].str.replace(',', '').astype(float)


In [120]:
df['SCAPITAL'] = df['SCAPITAL'].str.replace(',', '').astype(float)

In [16]:
query = "SELECT NOMBRE_ESTADO, COUNT(*) FROM df GROUP BY NOMBRE_ESTADO"

In [17]:
df2=sqldf(query)
df2

,NOMBRE_ESTADO,COUNT(*)
0,BAJA CALIFORNIA,277
1,BAJA CALIFORNIA SUR,1
2,DISTRITO FEDERAL,4
3,GUERRERO,27
4,HIDALGO,1931
5,MEXICO,6
6,MORELOS,83
7,OAXACA,1632
8,PUEBLA,5467
9,SAN LUIS POTOSI,177


In [57]:
query="""WITH socio_ranking AS (
  SELECT
    SUC,
    SOCIO,
    CRED,
    MONTO,
    DENSE_RANK() OVER (ORDER BY MONTO DESC) as ranking
  FROM df
)
SELECT
   SUC,
   SOCIO,
   CRED,
   MONTO,
    ranking
FROM socio_ranking
WHERE ranking BETWEEN 2 AND  4
ORDER BY ranking asc"""

In [58]:
df3=sqldf(query)
df3

,SUC,SOCIO,CRED,MONTO,ranking
0,205,"1,650","2,518",210000.0,2
1,1,216,210,200000.0,3
2,1,244,205,200000.0,3
3,1,508,262,200000.0,3
4,1,552,282,200000.0,3
5,1,584,294,200000.0,3
6,1,589,299,200000.0,3
7,1,602,307,200000.0,3
8,1,606,309,200000.0,3
9,2,29,"3,232",200000.0,3


In [88]:
query = "SELECT NOMBRE_ESTADO, COUNT(*), CAST(SUM(MONTO)/1000000 AS  DECIMAL(10,2)) FROM df GROUP BY NOMBRE_ESTADO ORDER BY 2 DESC"

In [89]:
df4=sqldf(query)
df4

,NOMBRE_ESTADO,COUNT(*),"CAST(SUM(MONTO)/1000000 AS DECIMAL(10,2))"
0,PUEBLA,5467,107.272674
1,HIDALGO,1931,40.171260
2,OAXACA,1632,39.897325
3,VERACRUZ DE IGNACIO DE LA LLAVE,506,8.733000
4,BAJA CALIFORNIA,277,7.580000
5,SAN LUIS POTOSI,177,3.070000
6,MORELOS,83,6.575000
7,GUERRERO,27,1.250000
8,MEXICO,6,0.191000
9,DISTRITO FEDERAL,4,0.168900


In [100]:
query = """
SELECT
NOMBRE_ESTADO,
  CASE
    WHEN SCAPITAL <= 20000 THEN 'low'
    WHEN SCAPITAL > 20000 AND SCAPITAL <= 70000 THEN 'medium'
    WHEN SCAPITAL > 70000 THEN 'high'
  END AS saldo_total_category,
  COUNT(*) AS number_of_ctas
FROM    df
GROUP BY
NOMBRE_ESTADO,
  CASE
    WHEN SCAPITAL <= 20000 THEN 'low'
    WHEN SCAPITAL > 20000 AND SCAPITAL <= 70000 THEN 'medium'
    WHEN SCAPITAL > 70000 THEN 'high'
END
ORDER BY 2, 3 DESC
;
"""

In [101]:
df5=sqldf(query)
df5

,NOMBRE_ESTADO,saldo_total_category,number_of_ctas
0,MORELOS,high,37
1,PUEBLA,high,32
2,OAXACA,high,23
3,HIDALGO,high,12
4,BAJA CALIFORNIA,high,8
5,VERACRUZ DE IGNACIO DE LA LLAVE,high,4
6,DISTRITO FEDERAL,high,1
7,GUERRERO,high,1
8,PUEBLA,low,4728
9,HIDALGO,low,1653


In [105]:
df['DESCRIPCION_PRODUCTO'].unique()

array(['CREDITO COLABORADORES', 'CREDITO AGROPECUARIO SOLIDARIO',
       'CREDITO GARANTIZADO SOBRE INVERSION',
       'CREDITO PRODUCTIVO ESTRATEGICO 1',
       'CREDITO AGROPECUARIO INDIVIDUAL', 'CREDITO NEGOCIO',
       'CREDITO PARA EMPLEADOS Y DIRECTIVOS', 'CREDITO MEJORA LA CASA',
       'CREDITO MI CASA AUTOPRODUCCION', 'CREDITO DE CONSUMO',
       'CREDITO NEGOCIO SOLIDARIO',
       'CREDITO MI CASA AUTOPRODUCCION PROTOTIPO',
       'CREDITO MI CASA MEJORAMIENTO',
       'CREDITO EMPRENDEDOR Y MICROEMPRESARIAL', 'CREDITO EMPRESARIAL'],
      dtype=object)

In [131]:
query = """

SELECT      NOMBRE_ESTADO,
       DESCRIPCION_PRODUCTO,
       SUM(MONTO) MONTO
FROM     df

GROUP BY NOMBRE_ESTADO,
       DESCRIPCION_PRODUCTO
HAVING   CASE 
            WHEN DESCRIPCION_PRODUCTO LIKE '%AGRO%'
                THEN SUM(MONTO) > 50000
            WHEN DESCRIPCION_PRODUCTO LIKE '%CONSU%'
                THEN SUM(MONTO) > 50000
            
            ELSE  SUM(MONTO)> 7000000 
   END;
"""

In [132]:
df6=sqldf(query)
df6

,NOMBRE_ESTADO,DESCRIPCION_PRODUCTO,MONTO
0,BAJA CALIFORNIA,CREDITO AGROPECUARIO INDIVIDUAL,420000.00
1,BAJA CALIFORNIA,CREDITO AGROPECUARIO SOLIDARIO,660000.00
2,HIDALGO,CREDITO AGROPECUARIO INDIVIDUAL,560000.00
3,HIDALGO,CREDITO AGROPECUARIO SOLIDARIO,180000.00
4,HIDALGO,CREDITO DE CONSUMO,2189000.00
5,HIDALGO,CREDITO MEJORA LA CASA,14531000.00
6,HIDALGO,CREDITO NEGOCIO,13930259.58
7,HIDALGO,CREDITO NEGOCIO SOLIDARIO,8353000.00
8,OAXACA,CREDITO AGROPECUARIO INDIVIDUAL,2939000.00
9,OAXACA,CREDITO AGROPECUARIO SOLIDARIO,1577000.00


In [117]:
query = "SELECT DESCRIPCION_PRODUCTO, COUNT(*), CAST(SUM(MONTO) AS  DECIMAL(10,2)) FROM df GROUP BY DESCRIPCION_PRODUCTO ORDER BY 2 DESC"

In [121]:
df7=sqldf(query)
df7

,DESCRIPCION_PRODUCTO,COUNT(*),"CAST(SUM(MONTO) AS DECIMAL(10,2))"
0,CREDITO NEGOCIO,3899,78609259.58
1,CREDITO MEJORA LA CASA,2846,62377983.60
2,CREDITO NEGOCIO SOLIDARIO,2600,44332925.13
3,CREDITO DE CONSUMO,282,5097195.77
4,CREDITO AGROPECUARIO INDIVIDUAL,199,9124515.66
5,CREDITO PRODUCTIVO ESTRATEGICO 1,115,8155000.00
6,CREDITO AGROPECUARIO SOLIDARIO,60,3169080.00
7,CREDITO GARANTIZADO SOBRE INVERSION,30,676000.00
8,CREDITO COLABORADORES,30,488200.00
9,CREDITO PARA EMPLEADOS Y DIRECTIVOS,21,789000.00


In [133]:
cap = pd.read_csv('./Captacion29122022.csv', encoding='UTF-8' )
cap.head(3)

,SUC,SOCIO,CUENTA,Producto,F. Inicio,F. Venc,% Int,SALDO
0,1,1,2,AHORRO CORRIENTE,19/01/18,NaN,5.0,556.87
1,1,2,2,AHORRO CORRIENTE,13/03/18,NaN,5.0,371.35
2,1,3,2,AHORRO CORRIENTE,08/11/18,NaN,5.0,404.06


In [134]:
cap.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 27409 entries, 0 to 27408
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   SUC        27409 non-null  int64  
 1   SOCIO      27409 non-null  object 
 2   CUENTA     27409 non-null  object 
 3   Producto   27409 non-null  object 
 4   F. Inicio  27409 non-null  object 
 5   F. Venc    1607 non-null   object 
 6   % Int      27409 non-null  float64
 7   SALDO      27409 non-null  object 
dtypes: float64(1), int64(1), object(6)
memory usage: 1.7+ MB


In [136]:
cap['SALDO'] = cap['SALDO'].str.replace(',', '').astype(float)

In [137]:
df.columns

Index(['SUC', 'SOCIO', 'NOMBRE_DEL_SOCIO', 'F. NACIMIENTO', 'SEXO',
       'NOMBRE_ESTADO', 'NOMBRE_MUNICIPIO', 'NOMBRE_COLONIA',
       'NOMBRE_LOCALIDAD', 'POBLACION', 'F. EMISION', 'MONTO', 'F. VENCE',
       'T. INT', 'N. PAG', 'F. PAG', 'C VIGENTE', 'C VENCIDO', 'SCAPITAL',
       'D. VEN', 'D X VEN', 'INTERES', 'MORA', 'DESCRIPCION_PRODUCTO',
       'NOMBRE PROMOTOR', 'CRED', 'DESTINO'],
      dtype='object')

In [138]:
cap.columns

Index(['SUC', 'SOCIO', 'CUENTA', 'Producto', 'F. Inicio', 'F. Venc', '% Int',
       'SALDO'],
      dtype='object')

In [145]:
query = """
SELECT CP.SUC, CP.SOCIO, CP.CUENTA, CP.SALDO, CR.CRED, CR.MONTO  
FROM df CR 
INNER JOIN cap CP 
    ON (CP.SUC=CR.SUC AND CP.SOCIO=CR.SOCIO)
WHERE CP.SALDO > 10000 AND CR.MONTO>50000    

"""
query

'\nSELECT CP.SUC, CP.SOCIO, CP.CUENTA, CP.SALDO, CR.CRED, CR.MONTO  \nFROM df CR \nINNER JOIN cap CP \n    ON (CP.SUC=CR.SUC AND CP.SOCIO=CR.SOCIO)\nWHERE CP.SALDO > 10000 AND CR.MONTO>50000    \n\n'

In [146]:
df8=sqldf(query)
df8

,SUC,SOCIO,CUENTA,SALDO,CRED,MONTO
0,1,474,2,164323.15,247,160000.0
1,1,553,2,60842.59,283,60000.0
2,1,557,2,101369.98,287,100000.0
3,1,558,2,60834.13,288,60000.0
4,1,559,2,60834.13,289,60000.0
5,1,587,2,101285.58,297,100000.0
6,2,50,3,182582.31,"3,216",60000.0
7,3,51,2,108100.87,"1,429",80000.0
8,3,381,471,10154.51,"1,547",70000.0
9,4,135,2,33527.50,359,90000.0
